Protest activity has increased worldwide since the death of George Floyd. Also, data collection on these events has improved and there are several sources available with basic data. This notebook is a high-level EDA of some of the sources.

# Locations

Below is a global map of protest activity. As described by the author:
> "Every town or city I can find where a George Floyd / Black Lives Matter protest, action, or vigil has occurred since May 25, 2020"

In [ ]:
import pandas as pd
import colorcet as cc
import geoviews as gv
import holoviews as hv
from holoviews.operation.datashader import datashade, dynspread
hv.extension('bokeh')

In [ ]:
protests = pd.read_json('../input/protests-against-police-violence/global_protests.json', orient='values')
df = pd.json_normalize(protests.features)
df.head()

The map below is interactive with one key limitation. The notebook has to be in edit mode, or "live", for the data shading and zooming to work. The picture below the chart shows what a close zoom looks like. It's a 100km x 100km area around where I live.

My house is near the light blue dot :)

In [ ]:
%%opts RGB {+axiswise} [width=650 height=620 xaxis=None yaxis=None] (alpha=0.8)

points_trn = gv.Points(df, kdims=['properties.X_Longitude', 'properties.Y_Latitude'])
spots_trn = datashade(points_trn, cmap=cc.fire, normalization='eq_hist') 
spots_trn = dynspread(spots_trn, threshold=1.0)

tiles = gv.tile_sources.CartoMidnight
me  = gv.Points(pd.DataFrame({'lon': -97.1895, 'lat': 32.9217}, index=[0])).opts()

spots_trn * tiles * me


![DFW](https://i.imgur.com/sAlXoW5.png)

<br>
Below is a map from another source, the ACLED US Crisis Monitor. Their database has over 11500 protests as of the end of August 2020.

In [ ]:
protests_us = pd.read_csv('../input/protests-against-police-violence/USA_2020_Aug29_ACLED.csv')
protests_us.columns = protests_us.columns.str.lower()
protests_us['event_date'] = pd.to_datetime(protests_us.event_date).dt.strftime('%m-%d')
protests_us

In [ ]:
%%opts Points {+axiswise} [width=650 height=620 xaxis=None yaxis=None] (alpha=0.2)

points = gv.Points(protests_us, kdims=['longitude', 'latitude'])
tiles = gv.tile_sources.CartoLight

points*tiles

This map doesn't show the hotspots as well, but you can zoom and pan when viewing the notebook.

# Timing

Here's a quick look at how protests unfolded over time in the US. It looks like protests started decreasing in June and leveled off. The small spikes are mostly for Saturday nights. The bigger spike on 8/22 was likely due to a reaction to the night before when 5 policemen in Louisiana shot a black man armed with a knife.

In [ ]:
x_ticks = [(8, 'June'), (38, 'July'), (69, 'Aug'), (90, '22Aug')]  # not working??

chart_opts = {'width': 650,
              'tools': ['hover'],
              'xlabel': "Date",
              'xticks': x_ticks,
              'ylabel': "Protests",
              'yticks': list(range(0, 701, 100))
              }


events_peace = ['Protests','Strategic developments']
sub_events_peace = ['Peaceful protest', 'Change to group/activity', 'Other']


def show_counts(df):
    df_piece = df.loc[:, 'event_date'] \
                 .value_counts() \
                 .sort_index() \
                 .rename("counts") \
                 .reset_index()
    plot = hv.Curve(df_piece).opts(**chart_opts)
    return df_piece, plot

peace_df = protests_us[protests_us.event_type.isin(events_peace) & \
                       protests_us.sub_event_type.isin(sub_events_peace)]

nopeace_df = protests_us.drop(peace_df.index)

piece_peace, p = show_counts(peace_df)
piece_nopeace, np = show_counts(nopeace_df)

p * np

Here's a more detailed view of the proportion of violent activity.

In [ ]:
chart_opts = {'width': 650,
              'tools': ['hover'],
              'xlabel': "Date",
              'xticks': x_ticks,
              'ylabel': "Proportion of all Activity",
              }

props = piece_nopeace.counts / (piece_nopeace.counts + piece_peace.counts)
hv.Curve(props).opts(**chart_opts)

All for now...